STEPS

1.	Create csv file for one directory (smile/neutral)
2.	Read txt file and extract timestamps, phonemes
3.	Populate database
4.	Define functions that measures formants and populates database
5.  eliminate non vowels
6.  streamline, make into functions
7.  do all audios of this directory



In [1]:
#1
import csv
import pandas as pd
import numpy as np

with open("smile_db.csv","w") as f:
    write = csv.writer(f)


In [2]:
#2
stamps = []
with open("smile/lab/SpkBeng_0001.lab","r") as text:
    lines = text.readlines()
for line in lines:
    stamps.append(line.split())

for i in stamps:
    i[0] = int(i[0])/10000000
    i[1] = int(i[1])/10000000 
    
#add time of measurement 
for i in stamps:
    middle = ((i[1]-i[0])/2)+i[0]
    i.append(middle)
    
        

In [3]:
#3
with open("smile_db.csv","w") as f:
    write = csv.writer(f)
    write.writerows(stamps)

In [4]:
#add headers
smile_db = pd.read_csv("smile_db.csv")
smile_db.columns = ["initial_time","end_time","phoneme","middle_point"]

In [5]:
#4 http://blog.syntheticspeech.de/2021/03/10/how-to-extract-formant-tracks-with-praat-and-python/
import parselmouth 
from parselmouth import praat
from parselmouth.praat import call

testfile = '/Users/julietaloizaga/OneDrive - Queen Mary, University of London/Documents/Coding4Lin/Final Project/smile/wav/SpkBeng_0001.wav'
sound = parselmouth.Sound(testfile) 
formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)



f1_list = []
f2_list = []
f3_list = []

for row in smile_db.index:
    
    t = smile_db.iat[row,3]
    
    f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
    f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
    f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
    
    f1_list.append(int(f1))
    f2_list.append(int(f2))
    f3_list.append(int(f3))
    

In [6]:
formant_frame = pd.DataFrame({'f1': f1_list, 'f2': f2_list, 'f3': f3_list})
smile_db = smile_db.join(formant_frame)


In [7]:
#get pitch 
pitch = call(sound, "To Pitch", 0.0, 25, 500) 

pitch_list = []

for row in smile_db.index:
    
    t = smile_db.iat[row,3]
    get_pitch = call(pitch, "Get value at time",t, 'Hertz', 'Linear')
    
    pitch_list.append(get_pitch)
    
pitch_list
    
#join to main DF
smile_db = smile_db.join(pd.DataFrame({"f0":pitch_list}))


In [8]:
#all the silences
index = smile_db.index
silence = smile_db["phoneme"] == "sil"
sil_ind = index[silence]
sil_ind_list = sil_ind.tolist()


In [9]:
#all the consonants
con_ind_list = []

def is_consonant(row):
    vowels = ["a","e","i","o","u"]
    for vowel in vowels:
        if vowel in smile_db.iat[row,2]:
            cons = False
            return cons
        else:
            cons = True
    
    return cons

for row in smile_db.index:
    if is_consonant(row) == True:
        con_ind_list.append(row)
        

In [10]:
#indices to delete
delete_indices = sil_ind_list + con_ind_list
delete_indices.sort()
smile_db = smile_db.drop(delete_indices)

smile_db_ri = smile_db.reset_index()
smile_db = smile_db_ri.drop("index",axis=1)


In [11]:
#add duration
dur_list = []

for i in smile_db.index:
    duration = smile_db.iat[i,1]-smile_db.iat[i,0]
    dur_list.append(duration)
    
    duration_df = pd.DataFrame({'vowel_length':dur_list})
    
smile_db = smile_db.join(duration_df)
    

In [12]:

smile_db

,initial_time,end_time,phoneme,middle_point,f1,f2,f3,f0,vowel_length
0,0.21,0.28,ao,0.245,563,1168,2606,148.907154,0.07
1,0.42,0.55,er,0.485,458,1371,2138,158.699590,0.13
2,0.55,0.59,ah,0.570,457,1212,2145,154.481419,0.04
3,0.70,0.77,ax,0.735,361,1508,2339,147.537175,0.07
4,0.85,0.97,ey,0.910,436,1750,2451,159.701881,0.12
5,1.10,1.20,er,1.150,344,1546,2298,155.022458,0.10
6,1.40,1.54,ey,1.470,472,1605,2294,140.957492,0.14
7,2.02,2.08,ih,2.050,396,1457,2189,172.663523,0.06
8,2.14,2.20,ax,2.170,407,1483,2184,162.256227,0.06
9,2.50,2.69,iy,2.595,307,1791,2396,156.358687,0.19
